In [3]:
import Constants as c           # constants
import polars as pl             # polars
#import os                       # for reading local files and folders
#import zipfile                  # for handling zip files
#from collections import Counter # for stats analysis on the dataframe  
#import math                     # for math functions
#import matplotlib.pyplot as plt # for plotting
#import statistics as stats      # for statistics functions

### Denmark Dataset
Import data from a completely new dataset (already "cleaned") to test the algorithm in a new scenario

In [4]:
filename='denmark_2023-06-05_point-based.parquet'
df = pl.read_parquet(c.FOLDER_PATH + filename)

In [5]:
# Rename columns to the same naming scheme used in the Hawaii DataFrame
display(df.head(1))
new_names = {
                "t":"BaseDateTime", "transtype":"TransceiverClass", "vid":"MMSI", "lat":'LAT', 'lon':'LON', 'navstatus':'Status', 'rot':'ROT',
                'sog':'SOG', 'cog':'COG', 'heading':'Heading', 'imo':'IMO', 'callsign':'CallSign', 'name':'VesselName',
                'shiptype':'VesselType', 'cargotype':'Cargo', 'width':'Width', 'length':'Length', 'gpstype':'GPSType', 
                'draught':'Draft', 'dest':'Destination', 'eta':'ETA', 'datasource':'Datasource'
            }
df = df.rename(new_names)
display(df.head(1))

t,transtype,vid,lat,lon,navstatus,rot,sog,cog,heading,imo,callsign,name,shiptype,cargotype,width,length,gpstype,draught,dest,eta,datasource,A,B,C,D,t_1,latr,lonr,havr_dist,speed_calc,journey,t_2,report_rate,heading_custom,rot_avg,rot_diff,speed_avg,speed_diff,__index_level_0__
datetime[ns],str,i64,f64,f64,str,f64,f64,f64,f64,str,str,str,str,str,f64,f64,str,f64,str,str,str,f64,f64,f64,f64,duration[ns],f64,f64,f64,f64,i64,duration[ns],f64,f64,f64,f64,f64,f64,i64
2023-05-06 13:11:54,"""Class A""",205096000,55.6581,5.47449,"""Under way using engine""",null,2.7,291.0,298.0,"""9898668""","""OPAS""","""Z19 BRIGITTE""","""Fishing""",null,8.0,34.0,"""GPS""",5.2,"""ZEEBRUGGE""","""26/04/2024 13:30:00""","""AIS""",12.0,22.0,6.0,2.0,null,0.971417,0.095548,null,null,35,null,null,null,null,null,null,null,1299


BaseDateTime,TransceiverClass,MMSI,LAT,LON,Status,ROT,SOG,COG,Heading,IMO,CallSign,VesselName,VesselType,Cargo,Width,Length,GPSType,Draft,Destination,ETA,Datasource,A,B,C,D,t_1,latr,lonr,havr_dist,speed_calc,journey,t_2,report_rate,heading_custom,rot_avg,rot_diff,speed_avg,speed_diff,__index_level_0__
datetime[ns],str,i64,f64,f64,str,f64,f64,f64,f64,str,str,str,str,str,f64,f64,str,f64,str,str,str,f64,f64,f64,f64,duration[ns],f64,f64,f64,f64,i64,duration[ns],f64,f64,f64,f64,f64,f64,i64
2023-05-06 13:11:54,"""Class A""",205096000,55.6581,5.47449,"""Under way using engine""",null,2.7,291.0,298.0,"""9898668""","""OPAS""","""Z19 BRIGITTE""","""Fishing""",null,8.0,34.0,"""GPS""",5.2,"""ZEEBRUGGE""","""26/04/2024 13:30:00""","""AIS""",12.0,22.0,6.0,2.0,null,0.971417,0.095548,null,null,35,null,null,null,null,null,null,null,1299


In [6]:
df = df.drop(['ROT', 'GPSType', 'Destination', 'ETA', 'Datasource', 'A', 'B', 'C', 'D', 't_1', 'latr', 'lonr'])
df = df.drop(['havr_dist', 'speed_calc', 'journey', 't_2', 'report_rate', 'heading_custom', 'rot_avg', 'rot_diff'])
df = df.drop(['speed_avg', 'speed_diff'])
#display(df.group_by("VesselType").agg(pl.len()))
pl.enable_string_cache()
df = df.with_columns(
                        pl.col('VesselType').cast(pl.Categorical),
                        pl.col('Status').cast(pl.Categorical),
                        pl.col('Cargo').cast(pl.Categorical)
                    )
display(df.head(1))

BaseDateTime,TransceiverClass,MMSI,LAT,LON,Status,SOG,COG,Heading,IMO,CallSign,VesselName,VesselType,Cargo,Width,Length,Draft,__index_level_0__
datetime[ns],str,i64,f64,f64,cat,f64,f64,f64,str,str,str,cat,cat,f64,f64,f64,i64
2023-05-06 13:11:54,"""Class A""",205096000,55.6581,5.47449,"""Under way using engine""",2.7,291.0,298.0,"""9898668""","""OPAS""","""Z19 BRIGITTE""","""Fishing""",null,8.0,34.0,5.2,1299


In [7]:
# check how many unique elements for each of this columns that have been converted to 'categorical'
display(len(df.group_by("VesselType").agg(pl.len())))
display(len(df.group_by("Status").agg(pl.len())))
display(len(df.group_by("Cargo").agg(pl.len())))

25

14

7

In [8]:
display(len(df.group_by("TransceiverClass").agg(pl.len())))
# Since the column 'TransceiverClass' can only contain two values ('Class A' and 'Class B')
# change it to a boolean column where True means that the value is 'Class A'
df = df.with_columns(
    IsClassA = (pl.col('TransceiverClass') == 'Class A')
).drop('TransceiverClass')
display(df.head(1))

1

BaseDateTime,MMSI,LAT,LON,Status,SOG,COG,Heading,IMO,CallSign,VesselName,VesselType,Cargo,Width,Length,Draft,__index_level_0__,IsClassA
datetime[ns],i64,f64,f64,cat,f64,f64,f64,str,str,str,cat,cat,f64,f64,f64,i64,bool
2023-05-06 13:11:54,205096000,55.6581,5.47449,"""Under way using engine""",2.7,291.0,298.0,"""9898668""","""OPAS""","""Z19 BRIGITTE""","""Fishing""",null,8.0,34.0,5.2,1299,true


In [9]:
display(f'Dataframe size in RAM {df.estimated_size()/1_000_000_000:.3f} GB')
# export final df as parquet file
df.write_parquet(file=c.FOLDER_PATH + 'cleaned_' + filename)

'Dataframe size in RAM 1.085 GB'